# NOTE: We will revise pytorch and implement a language model from scratch hopefully this will clear all doubts for future weeks

# Pytorch Basic Session :1

In [1]:
#Step 1 : Understanding nn.Module
import torch
import torch.nn as nn

class SimpleModule(nn.Module):
    def __init__(self):
        super().__init__()  #By calling super().__init__(), you're saying: "Hey parent class, set up all your infrastructure before I add my custom stuff."(parent class's constructor.))
        
    def forward(self, x):
        return x

In [2]:
#Setp 1:1 Example
import torch
import torch.nn as nn

class SimpleModule(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, x):
        return x * 2

# Create model
model = SimpleModule()

# Test it
x = torch.tensor([1.0, 2.0, 3.0])
output = model(x)  # This calls forward() automatically

print(f"Input: {x}")
print(f"Output: {output}")

Input: tensor([1., 2., 3.])
Output: tensor([2., 4., 6.])


In [ ]:
#Step 2 :Understanding nn.Parameter
import torch
import torch.nn as nn

class ModuleWithWeight(nn.Module):
    def __init__(self, size):
        super().__init__()
        
        # This creates a LEARNABLE parameter
        self.weight = nn.Parameter(torch.randn(size))
        
    def forward(self, x):
        return x * self.weight

# Create model
model = ModuleWithWeight(3)

# Test
x = torch.tensor([1.0, 2.0, 3.0])
output = model(x)

print(f"Input: {x}")
print(f"Weight: {model.weight}")
print(f"Output: {output}")
print(f"\nIs weight learnable? {model.weight.requires_grad}")

Input: tensor([1., 2., 3.])
Weight: Parameter containing:
tensor([-1.1037,  0.4698, -0.1627], requires_grad=True)
Output: tensor([-1.1037,  0.9396, -0.4880], grad_fn=<MulBackward0>)

Is weight learnable? True


In [4]:
# Step 3 : Understanding nn.Linear
import torch
import torch.nn as nn

# Create a linear layer
input_dim = 512
output_dim = 256
linear = nn.Linear(input_dim, output_dim)

# What does it contain?
print(f"Weight shape: {linear.weight.shape}")
print(f"Bias shape: {linear.bias.shape}")
print(f"Weight requires_grad: {linear.weight.requires_grad}")

Weight shape: torch.Size([256, 512])
Bias shape: torch.Size([256])
Weight requires_grad: True


In [ ]:
import torch
import torch.nn as nn

# Create linear layer
linear = nn.Linear(512, 256)
# This creates a weight matrix of 256 and 512

# Create input
batch_size = 32
seq_len = 10
x = torch.randn(batch_size, seq_len, 512)

# Forward pass
output = linear(x)

print(f"Input shape:  {x.shape}")      # [32, 10, 512]
print(f"Output shape: {output.shape}")  # [32, 10, 256]

Input shape:  torch.Size([32, 10, 512])
Output shape: torch.Size([32, 10, 256])


# Buliding Token Embeddings

In [1]:
import torch
import torch.nn as nn

#Start by inheriting from nn.Module
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super().__init__() #Call the parent class's constructor so that all methods of nn.Module are properly initialized.
        #nn.embedding creates a lookup table that maps from integer indices (representing tokens) to dense vectors of fixed size (the embeddings).
        #nn.embedding ,creates a lookup table and intializes it with random values. and makes the values learnable parameters of the model. (also registers it as a parameter of the module)
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        
    def forward(self, x):
        return self.embedding(x)
    
#Create
vocab_size = 1000  # Size of the vocabulary
d_model = 512    # Dimension of the embeddings
token_embed= TokenEmbedding(vocab_size, d_model)

#Test
token_ids=torch.randint(0, vocab_size, (2,10))
output= token_embed (token_ids)
print(f"Input token IDs shape: {token_ids.shape}")  # [2, 10]
print(f"Output embeddings shape: {output.shape}")    # [2, 10,

Input token IDs shape: torch.Size([2, 10])
Output embeddings shape: torch.Size([2, 10, 512])


# Position embeddings

In [ ]:
import torch
import torch.nn as nn

class PositionEmbedding(nn.Module): #Inherit from nn.Module
    def __init__(self, max_seq_len, d_model):
        super().__init__() #Call the parent class's constructor to make sure all methods of nn.Module are properly initialized. and available.
        #NOTE:Transformers (Original paper) used sinusoidal position embeddings, but nn.Embedding is a common choice for learnable position embeddings.(we will make sure gradients also flow through these embeddings during training.)
        #This creates a learnable position embedding table of size (max_seq_len, d_model)
        self.position_embedding = nn.Embedding(max_seq_len, d_model)

    def forward(self, seq_len):
        #Generate position indices from 0 to seq_len - 1
        positions=torch.arange(seq_len)#This creaates a tensor with seqential numbersye
        return self.position_embedding(positions)
    
#Create
pos_embed= PositionEmbedding(max_seq_len=2048, d_model=512)
        
output = pos_embed(10)  # Example input sequence length
print(f"Output position embeddings shape: {output.shape}")  # [10, 512]
print(f"Positions Used: {torch.arange(10)}")

Output position embeddings shape: torch.Size([10, 512])
Positions Used: tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


# Combine Token and Position Embeddings

In [ ]:
import torch
import torch.nn as nn

class Embeddings(nn.Module):#This is normal inheritance from nn.Module
    def __init__(self, vocab_size, max_seq_len, d_model):
        super().__init__() #Initialize the parent class nn.Module so that all its methods and attributes are available. and usable.
        """
        1.token embedding layer to convert token IDs into dense vectors.
        2.position embedding layer to add positional information to the token embeddings.

        NOTE:Both embeddings are learnable parameters of the model.
        """
        
        self.token_embed = nn.Embedding(vocab_size, d_model)
        self.pos_embed = nn.Embedding(max_seq_len, d_model)
        
    def forward(self, token_ids):
        batch_size, seq_len = token_ids.shape
        
        tokens = self.token_embed(token_ids)
        #torch.arrange helps to convert tokens into id's which can be used to get position embeddings from nn.Embedding lookup table.
        positions = torch.arange(seq_len, device=token_ids.device)
        pos = self.pos_embed(positions)
        
        return tokens + pos

# Test
embeddings = Embeddings(vocab_size=1000, max_seq_len=2048, d_model=512)
token_ids = torch.randint(0, 1000, (2, 10))  # [2, 10]
output = embeddings(token_ids)

print(f"Input shape: {token_ids.shape}")
print(f"Output shape: {output.shape}")

Input shape: torch.Size([2, 10])
Output shape: torch.Size([2, 10, 512])
